# Deep Learning
## Assignment 3
Previously in 2_fullyconnected.ipynb, you trained a logistic regression and a neural network model.
The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in 1_notmnist.ipynb.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
* data as a flat matrix,
* labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## Problem 1
Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy.

### Nerual Networks

In [8]:
batch_size = 128
hidden_nodes = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_hidden = tf.Variable(tf.zeros([hidden_nodes]))
    
  weights = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)
  logits = tf.matmul(layer1, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))\
        +beta*tf.nn.l2_loss(weights_hidden) +\
        +beta*tf.nn.l2_loss(biases_hidden) +\
        +beta*tf.nn.l2_loss(weights) +\
        +beta*tf.nn.l2_loss(biases)
    
 # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
  valid_prediction = tf.nn.softmax(tf.matmul(layer1_valid, weights) + biases)
    
  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
  test_prediction  = tf.nn.softmax(tf.matmul(layer1_test, weights)  + biases)

Let's run it:

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3482.053711
Minibatch accuracy: 11.7%
Validation accuracy: 25.8%
Minibatch loss at step 500: 21.358454
Minibatch accuracy: 85.2%
Validation accuracy: 83.6%
Minibatch loss at step 1000: 0.863870
Minibatch accuracy: 85.9%
Validation accuracy: 81.9%
Minibatch loss at step 1500: 0.625290
Minibatch accuracy: 88.3%
Validation accuracy: 83.1%
Minibatch loss at step 2000: 0.698040
Minibatch accuracy: 85.9%
Validation accuracy: 82.4%
Minibatch loss at step 2500: 0.746145
Minibatch accuracy: 82.8%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 0.825475
Minibatch accuracy: 79.7%
Validation accuracy: 83.2%
Test accuracy: 89.9%


## Problem 2
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?


In [11]:
batch_size = 32
hidden_nodes = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_hidden = tf.Variable(tf.zeros([hidden_nodes]))
    
  weights = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)
  logits = tf.matmul(layer1, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))\
        +beta*tf.nn.l2_loss(weights_hidden) +\
        +beta*tf.nn.l2_loss(biases_hidden) +\
        +beta*tf.nn.l2_loss(weights) +\
        +beta*tf.nn.l2_loss(biases)
    
 # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
  valid_prediction = tf.nn.softmax(tf.matmul(layer1_valid, weights) + biases)
    
  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
  test_prediction  = tf.nn.softmax(tf.matmul(layer1_test, weights)  + biases)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3553.341064
Minibatch accuracy: 6.2%
Validation accuracy: 26.4%
Minibatch loss at step 500: 22.480213
Minibatch accuracy: 81.2%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 0.908897
Minibatch accuracy: 87.5%
Validation accuracy: 78.8%
Minibatch loss at step 1500: 0.821943
Minibatch accuracy: 84.4%
Validation accuracy: 74.3%
Minibatch loss at step 2000: 1.153963
Minibatch accuracy: 75.0%
Validation accuracy: 77.7%
Minibatch loss at step 2500: 1.033158
Minibatch accuracy: 71.9%
Validation accuracy: 79.3%
Minibatch loss at step 3000: 0.860388
Minibatch accuracy: 75.0%
Validation accuracy: 79.4%
Test accuracy: 86.5%


## Problem 3
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

In [76]:
batch_size = 128
layer1_nodes = 1024
layer2_nodes = 50
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels =  tf.placeholder(tf.float32,shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset =  tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([image_size * image_size, layer1_nodes]))
  layer1_biases = tf.Variable(tf.zeros([layer1_nodes]))
    
  layer2_weights = tf.Variable(tf.truncated_normal([layer1_nodes, layer2_nodes]))
  layer2_biases = tf.Variable(tf.zeros([layer2_nodes]))
    
  weights = tf.Variable(tf.truncated_normal([layer2_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
#   keep_prob = tf.placeholder(tf.float32)
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, layer1_weights) + layer1_biases)
#   layer1_drop = tf.nn.dropout(layer1, keep_prob)

  layer2 = tf.nn.relu(tf.matmul(layer1, layer2_weights) + layer2_biases)

#   layer2 = tf.nn.relu(tf.matmul(layer1_drop, layer2_weights) + layer2_biases)
#   layer2_drop = tf.nn.dropout(layer2, keep_prob)

  logits = tf.matmul(layer2, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))\
        +beta*tf.nn.l2_loss(layer1_weights) +\
        +beta*tf.nn.l2_loss(layer1_biases) +\
        +beta*tf.nn.l2_loss(layer2_weights) +\
        +beta*tf.nn.l2_loss(layer2_biases) +\
        +beta*tf.nn.l2_loss(weights) +\
        +beta*tf.nn.l2_loss(biases)
        #         +beta*tf.nn.l2_loss(layer2_weights) +\
#         +beta*tf.nn.l2_loss(layer2_biases) +\

    
 # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, layer1_weights) + layer1_biases)
  layer2_valid = tf.nn.relu(tf.matmul(layer1_valid, layer2_weights) + layer2_biases)
  valid_prediction = tf.nn.softmax(tf.matmul(layer2_valid, weights) + biases)
    
  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, layer1_weights) + layer1_biases)
  layer2_test = tf.nn.relu(tf.matmul(layer1_test, layer2_weights) + layer2_biases)
  test_prediction  = tf.nn.softmax(tf.matmul(layer2_test, weights)  + biases)

In [77]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1249.856689
Minibatch accuracy: 10.9%
Validation accuracy: 10.2%
Minibatch loss at step 500: nan
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1000: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Minibatch loss at step 1500: nan
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 2000: nan
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 2500: nan
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 3000: nan
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Test accuracy: 10.0%


In [52]:
batch_size = 128
hidden_nodes = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_hidden = tf.Variable(tf.zeros([hidden_nodes]))
    
  weights = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

  keep_prob = tf.placeholder(tf.float32)
  layer1_drop = tf.nn.dropout(layer1, keep_prob)

  logits = tf.matmul(layer1_drop, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))\
        +beta*tf.nn.l2_loss(weights_hidden) +\
        +beta*tf.nn.l2_loss(biases_hidden) +\
        +beta*tf.nn.l2_loss(weights) +\
        +beta*tf.nn.l2_loss(biases)
    
 # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 500, 0.9, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
#   optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
  valid_prediction = tf.nn.softmax(tf.matmul(layer1_valid, weights) + biases)
    
  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
  test_prediction  = tf.nn.softmax(tf.matmul(layer1_test, weights)  + biases)

In [56]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob:0.8}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 3528.100830
Minibatch accuracy: 11.7%
Validation accuracy: 34.8%
Test accuracy: 37.4%
Minibatch loss at step 500: 1155.243530
Minibatch accuracy: 77.3%
Validation accuracy: 82.4%
Test accuracy: 89.3%
Minibatch loss at step 1000: 466.152313
Minibatch accuracy: 82.0%
Validation accuracy: 83.4%
Test accuracy: 90.2%
Minibatch loss at step 1500: 206.521606
Minibatch accuracy: 85.2%
Validation accuracy: 84.6%
Test accuracy: 91.3%
Minibatch loss at step 2000: 99.496696
Minibatch accuracy: 85.9%
Validation accuracy: 85.2%
Test accuracy: 91.9%
Minibatch loss at step 2500: 51.894802
Minibatch accuracy: 82.0%
Validation accuracy: 85.8%
Test accuracy: 92.2%
Minibatch loss at step 3000: 29.055792
Minibatch accuracy: 84.4%
Validation accuracy: 85.5%
Test accuracy: 92.2%
Minibatch loss at step 3500: 17.335646
Minibatch accuracy: 86.7%
Validation accuracy: 85.3%
Test accuracy: 92.0%
Minibatch loss at step 4000: 11.029345
Minibatch accuracy: 89.1%
Validation accura